In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL"
os.chdir(path)
os.listdir(path)
import os
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL


In [2]:
!pip install accelerate
!pip install transformers


In [3]:
import pandas as pd

# 替换以下路径为您数据集的实际路径
file_path = 'AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt'

import chardet

# 打开文件，读取一部分数据来检测编码
with open(file_path, 'rb') as file:
    raw_data = file.read(10000)  # 读取文件的前10000字节
    encoding_result = chardet.detect(raw_data)
    file_encoding = encoding_result['encoding']
    print(f"Detected encoding: {file_encoding}")

# 使用检测到的编码重新打开文件
try:
    with open(file_path, 'r', encoding=file_encoding) as file:
        content = file.readlines()
    print("File read successfully with detected encoding.")
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError with {file_encoding}: {e}")


Detected encoding: ascii
UnicodeDecodeError with ascii: 'ascii' codec can't decode byte 0x96 in position 1614: ordinal not in range(128)


In [4]:
with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
    content = file.readlines()
print("File read with ignoring errors.")


File read with ignoring errors.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

data = []
for item in content:
    split_item = item.split('|')
    data.append((split_item[0], split_item[1], split_item[-1]))
print (data[0:2])


[('AB', 'abortion', '_%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida 3, para 2-0-0-2, who presented to the Emergency Room complaining of increasing vaginal bleeding since approximately 6 a.m. The patient does have a known history of having had a missed AB. She had been followed at another clinic and was told that she had a missed AB shortly after Christmas. The patient at that time had been counseled to undergo a D&C and was even offered misoprostol to help complete a miscarriage, however, patient declined at that time to schedule a D&C or to take the misoprostol.\n'), ('AB', 'abortion', 'She is now bleeding quite heavily. Ultrasound this morning demonstrated a missed AB consistent with a 6 week pregnancy with blood clots in the uterine cavity, as well as continued bleeding from the cervical os. This is consistent with an incomplete AB. The patient presents now for a suction D&C. Medical history is negative. Surgical history is negative. CURRENT MEDICATIONS: Include prenatal vitamins.\n

In [6]:
from collections import Counter
data2_acronym_dict = Counter([item[0] for item in data])
# print (data2_acronym_dict)
print (len(data2_acronym_dict))


75


In [7]:
word_sense_dict = {}
for item in data:
    if word_sense_dict.get(item[0]):
        word_sense_dict[item[0]][item[1]] += 1
    else:
        word_sense_dict[item[0]] = Counter()
        word_sense_dict[item[0]][item[1]] = 1
print (word_sense_dict)


{'AB': Counter({'abortion': 345, 'blood group in ABO system': 137, 'type A, type B': 8, 'atrioventricular:AV': 2, 'ankle-brachial': 1, 'arteriovenous:AV': 1, 'X-ray finding': 1, 'MISTAKE:abduction': 1, 'antipyrine benzocaine': 1, 'arterial blood': 1, 'UNSURED SENSE': 1, 'NAME': 1}), 'VBG': Counter({'vertical banded gastroplasty': 299, 'venous blood gas': 201}), 'AC': Counter({'(drug) AC': 161, 'acromioclavicular': 158, 'adriamycin cyclophosphamide': 118, 'before meals': 42, 'assist control': 9, 'acetate': 4, 'angiotensin-converting enzyme:ACE': 3, 'abdominal circumference': 2, 'alternating current': 1, 'anticoagulation': 1, 'antecubital': 1}), 'ALD': Counter({'ad lib on demand': 407, 'adrenoleukodystrophy': 88, 'alanine aminotransferase:ALT': 3, 'left anterior descending:LAD': 1, 'acetyl lysergic acid diethylamide': 1}), 'AMA': Counter({'against medical advice': 444, 'advanced maternal age': 31, 'antimitochondrial antibody': 25}), 'ASA': Counter({'acetylsalicylic acid': 404, 'American 

In [8]:
no_word_sense_per_acronym = []
for key, value in word_sense_dict.items():
    no_word_sense_per_acronym.append(len(value))
no_word_sense_per_acronym = Counter(no_word_sense_per_acronym)
print (no_word_sense_per_acronym)


Counter({5: 12, 4: 12, 3: 11, 6: 10, 2: 9, 8: 7, 7: 3, 9: 3, 12: 2, 10: 2, 11: 1, 1: 1, 14: 1, 15: 1})


In [9]:
no_sentences_per_word_sense = []
for acronym, value in word_sense_dict.items():
    for word_sense, no_sentences in value.items():
        no_sentences_per_word_sense.append(no_sentences)
no_sentences_per_word_sense = Counter(no_sentences_per_word_sense)
print (no_sentences_per_word_sense)
sentence_len_per_word_sense = Counter([len(item[2]) for item in data])
# print (data2_sentence_dict)
print (f"max length:{max(sentence_len_per_word_sense.keys())}; min length:{min(sentence_len_per_word_sense.keys())}")


Counter({1: 106, 2: 42, 3: 23, 4: 13, 5: 12, 6: 8, 8: 6, 7: 5, 14: 5, 31: 4, 11: 4, 88: 3, 25: 3, 103: 3, 36: 3, 16: 3, 439: 3, 345: 2, 137: 2, 9: 2, 444: 2, 93: 2, 457: 2, 47: 2, 231: 2, 105: 2, 225: 2, 53: 2, 396: 2, 41: 2, 336: 2, 34: 2, 469: 2, 30: 2, 176: 2, 275: 2, 394: 2, 40: 2, 308: 2, 57: 2, 55: 2, 12: 2, 138: 2, 83: 2, 299: 1, 201: 1, 161: 1, 158: 1, 118: 1, 42: 1, 407: 1, 404: 1, 381: 1, 374: 1, 116: 1, 43: 1, 343: 1, 157: 1, 456: 1, 459: 1, 434: 1, 249: 1, 243: 1, 261: 1, 391: 1, 270: 1, 453: 1, 28: 1, 100: 1, 222: 1, 278: 1, 62: 1, 436: 1, 152: 1, 282: 1, 462: 1, 286: 1, 209: 1, 129: 1, 23: 1, 45: 1, 448: 1, 18: 1, 289: 1, 200: 1, 449: 1, 51: 1, 265: 1, 446: 1, 19: 1, 472: 1, 461: 1, 38: 1, 102: 1, 500: 1, 58: 1, 48: 1, 188: 1, 426: 1, 134: 1, 107: 1, 179: 1, 314: 1, 418: 1, 82: 1, 207: 1, 279: 1, 123: 1, 377: 1, 474: 1, 10: 1, 438: 1, 121: 1, 466: 1, 32: 1, 212: 1, 61: 1, 111: 1, 294: 1, 361: 1, 409: 1, 408: 1, 89: 1, 423: 1, 74: 1, 141: 1, 252: 1, 455: 1, 22: 1, 21: 1, 6

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader

# 加载预训练的BioBERT分词器
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

class ClinicalData(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # 将模型需要的tensor打包成字典返回
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # 确保标签是长整型
        return item

    def __len__(self):
        return len(self.labels)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# 打印前几个数据点查看结构
for i in range(5):
    print(data[i])


('AB', 'abortion', '_%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida 3, para 2-0-0-2, who presented to the Emergency Room complaining of increasing vaginal bleeding since approximately 6 a.m. The patient does have a known history of having had a missed AB. She had been followed at another clinic and was told that she had a missed AB shortly after Christmas. The patient at that time had been counseled to undergo a D&C and was even offered misoprostol to help complete a miscarriage, however, patient declined at that time to schedule a D&C or to take the misoprostol.\n')
('AB', 'abortion', 'She is now bleeding quite heavily. Ultrasound this morning demonstrated a missed AB consistent with a 6 week pregnancy with blood clots in the uterine cavity, as well as continued bleeding from the cervical os. This is consistent with an incomplete AB. The patient presents now for a suction D&C. Medical history is negative. Surgical history is negative. CURRENT MEDICATIONS: Include prenatal vitamins.\n')

In [12]:


from sklearn.model_selection import train_test_split

# 假设`data`是一个列表，每个元素是一个包含(text, label)的元组
# 从数据中提取标签和文本
labels, texts = zip(*[(item[1], item[2]) for item in data])

# 清理非字符串类型的标签
labels = [label for label in labels if isinstance(label, str)]

# 现在尝试创建映射
label_map = {label: idx for idx, label in enumerate(sorted(set(labels)))}

# 显示标签映射以确保其正确性
print("标签映射:", label_map)


# 将字符串标签转换为整数索引
labels_indexed = [label_map[label] for label in labels]

# 显示转换后的整数标签示例以确保其正确性
print("转换后的整数标签示例:", labels_indexed[:5])

# 首先分割训练集和剩余部分
texts_train, texts_temp, labels_train, labels_temp = train_test_split(
    texts, labels_indexed, test_size=0.2, random_state=42)

# 然后将剩余部分分割为验证集和测试集
texts_val, texts_test, labels_val, labels_test = train_test_split(
    texts_temp, labels_temp, test_size=0.5, random_state=42)


标签映射: {'(PO Box) C4': 0, '(class) IA': 1, '(complement) component 3': 2, '(complement) component 4': 3, '(cycle) IB': 4, '(device) MP': 5, '(device) PD': 6, '(diltiazem) DC': 7, '(drug) AC': 8, '(drug) DC': 9, '(drug) DT': 10, '(drug) ES': 11, '(drug) IT': 12, '(drug) MP': 13, '(drug) MS': 14, '(drug) NP': 15, '(drug) OR': 16, '(drug) PAC': 17, '(drug) PD': 18, '(drug) PR': 19, '(drug) RT': 20, '(grade) IA': 21, '(grade) IB': 22, '(stage) C3': 23, '(stage) IA': 24, '(stage) IB': 25, '(status) IA': 26, '(status) IB': 27, '(type) IA': 28, '(type) IB': 29, 'American Society of Anesthesiologists': 30, 'American Society of Anesthesiologists:ASA': 31, 'BK (virus)': 32, 'C-reactive': 33, 'California': 34, "Children's Depression Inventory": 35, 'District of Columbia': 36, 'Fairview Southdale Hospital': 37, 'GENERAL ENGLISH': 38, 'Iowa': 39, 'Los Angeles': 40, 'Louisiana': 41, 'MISTAKE:EZ PAP': 42, 'MISTAKE:Oncotype DX': 43, 'MISTAKE:abduction': 44, 'Mall of America:MOA': 45, 'NAME': 46, 'Narco

In [13]:
train_encodings = tokenizer(list(texts_train), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(texts_val), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(texts_test), truncation=True, padding=True, max_length=128)


In [14]:
from torch.utils.data import DataLoader, Dataset



# 更新你的数据集类，使用整数标签
class ClinicalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # 这应该是整数列表

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # 此处labels已经是整数
        return item

    def __len__(self):
        return len(self.labels)



        return len(self.labels)

# 创建数据集实例
train_dataset = ClinicalDataset(train_encodings, labels_train)
val_dataset = ClinicalDataset(val_encodings, labels_val)
test_dataset = ClinicalDataset(test_encodings, labels_test)



In [15]:
# 加载BioBERT模型
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=len(set(labels)))


from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录保存训练结果
    num_train_epochs=3,              # 训练轮数
    per_device_train_batch_size=8,   # 每个设备的训练批大小
    per_device_eval_batch_size=16,   # 每个设备的评估批大小
    warmup_steps=500,                # 预热步骤
    weight_decay=0.01,               # 权重衰减，帮助防止过拟合
    logging_dir='./logs',            # 日志目录
    logging_steps=500,                # 多少步打印一次日志
    evaluation_strategy="epoch",     # 每个epoch结束时评估模型性能
    save_strategy="epoch",           # 每个epoch保存模型
    load_best_model_at_end=True      # 训练结束时载入最佳模型
)

# 初始化Trainer
trainer = Trainer(
    model=model,                 # 模型实例
    args=training_args,          # 训练参数
    train_dataset=train_dataset, # 训练数据集
    eval_dataset=val_dataset,    # 验证数据集
)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.573900,0.481794
2,0.335400,0.412641
3,0.208000,0.384810


TrainOutput(global_step=11250, training_loss=0.7693890496148004, metrics={'train_runtime': 683.5415, 'train_samples_per_second': 131.667, 'train_steps_per_second': 16.458, 'total_flos': 5938549240320000.0, 'train_loss': 0.7693890496148004, 'epoch': 3.0})

In [17]:
# 保存训练好的模型
model_path = "./biobert_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./biobert_finetuned/tokenizer_config.json',
 './biobert_finetuned/special_tokens_map.json',
 './biobert_finetuned/vocab.txt',
 './biobert_finetuned/added_tokens.json')

In [18]:
print("Number of classes:", model.config.num_labels)


Number of classes: 351


In [36]:
print("Label Map:", label_map)
print("Inverse Label Map:", inverse_label_map)

# 检查预测标签索引是否全部在 inverse_label_map 中
pred_indices = set(np.argmax(predictions.predictions, axis=-1))
missing_indices = pred_indices - set(inverse_label_map.keys())
print("Missing Indices:", missing_indices)

if missing_indices:
    print("Warning: There are missing indices in the inverse_label_map!")
else:
    print("All predicted indices are accounted for in the inverse_label_map.")


Label Map: {0: 'Class_0', 1: 'Class_1', 2: 'Class_2', 3: 'Class_3', 4: 'Class_4', 5: 'Class_5', 6: 'Class_6', 7: 'Class_7', 8: 'Class_8', 9: 'Class_9', 10: 'Class_10', 11: 'Class_11', 12: 'Class_12', 13: 'Class_13', 14: 'Class_14', 15: 'Class_15', 16: 'Class_16', 17: 'Class_17', 18: 'Class_18', 19: 'Class_19', 20: 'Class_20', 21: 'Class_21', 22: 'Class_22', 23: 'Class_23', 24: 'Class_24', 25: 'Class_25', 26: 'Class_26', 27: 'Class_27', 28: 'Class_28', 29: 'Class_29', 30: 'Class_30', 31: 'Class_31', 32: 'Class_32', 33: 'Class_33', 34: 'Class_34', 35: 'Class_35', 36: 'Class_36', 37: 'Class_37', 38: 'Class_38', 39: 'Class_39', 40: 'Class_40', 41: 'Class_41', 42: 'Class_42', 43: 'Class_43', 44: 'Class_44', 45: 'Class_45', 46: 'Class_46', 47: 'Class_47', 48: 'Class_48', 49: 'Class_49', 50: 'Class_50', 51: 'Class_51', 52: 'Class_52', 53: 'Class_53', 54: 'Class_54', 55: 'Class_55', 56: 'Class_56', 57: 'Class_57', 58: 'Class_58', 59: 'Class_59', 60: 'Class_60', 61: 'Class_61', 62: 'Class_62', 

In [37]:
# 确定哪些索引没有在 inverse_label_map 中
missing_indices = set(predictions.predictions.argmax(axis=-1).flatten().tolist()) - set(inverse_label_map.keys())

# 如果有缺失的索引，可以手动添加它们到 inverse_label_map
for idx in missing_indices:
    if idx not in inverse_label_map:
        inverse_label_map[idx] = f"Class_{idx}"

print("Missing Indices Updated:", missing_indices)


Missing Indices Updated: {2, 3, 8, 21, 22, 24, 25, 30, 32, 35, 36, 37, 38, 39, 40, 47, 49, 51, 52, 55, 56, 59, 62, 63, 64, 65, 66, 67, 68, 69, 77, 82, 85, 88, 91, 92, 93, 94, 96, 97, 98, 101, 102, 103, 104, 106, 107, 109, 111, 112, 113, 115, 117, 119, 120, 121, 123, 126, 127, 128, 129, 130, 133, 135, 138, 140, 141, 142, 143, 146, 147, 148, 149, 151, 156, 157, 158, 159, 160, 162, 166, 168, 170, 171, 174, 176, 177, 179, 181, 183, 184, 186, 192, 193, 194, 195, 196, 197, 198, 200, 202, 204, 205, 207, 210, 217, 219, 220, 223, 224, 225, 228, 230, 232, 233, 234, 235, 238, 241, 243, 245, 247, 249, 250, 252, 253, 256, 259, 261, 262, 264, 265, 266, 267, 269, 270, 271, 272, 275, 276, 277, 278, 279, 280, 282, 283, 285, 287, 290, 292, 294, 297, 299, 302, 305, 308, 310, 311, 313, 316, 317, 319, 322, 323, 324, 325, 326, 330, 331, 332, 333, 334, 337, 338, 339, 340, 341, 344, 346, 347, 348, 350}


In [39]:
# 确保 inverse_label_map 包含所有可能的标签索引
inverse_label_map = {v: k for k, v in label_map.items()}

# 使用 DataLoader 加载测试数据集
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 遍历测试数据集
for batch in test_loader:
    # 将批次数据移动到模型所在的设备（例如：GPU）
    batch = {k: v.to(model.device) for k, v in batch.items()}

    # 模型评估模式
    model.eval()

    # 不计算梯度
    with torch.no_grad():
        outputs = model(**batch)

    # 获取预测结果的 logits 并找到概率最高的索引（预测的类别索引）
    pred_labels_idx = outputs.logits.argmax(-1).tolist()  # 转换为 Python 列表

    # 使用逆映射将索引转换为实际的标签
    predicted_labels = [inverse_label_map.get(idx, 'Unknown Label') for idx in pred_labels_idx]

    # 获取真实标签，这假设 'labels' 在批次数据中
    actual_labels = [inverse_label_map.get(label.item(), 'Unknown Label') for label in batch['labels']]

    # 打印预测结果和实际标签
    for pred_label, actual_label in zip(predicted_labels, actual_labels):
        print(f"Predicted Label: {pred_label}, Actual Label: {actual_label}")


Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual Label: Unknown Label
Predicted Label: Unknown Label, Actual L

KeyboardInterrupt: 